In [20]:
import spatialdata_io
import numpy as np
import pandas as pd
import scvi
import sys
import argparse
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import preprocessing

# Access the arguments
path = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/segger/NSCLC/lung/1GQ9/1GQ9/normalised_results/outs')
min_counts = 5
min_features = 5
max_counts = np.inf
max_features = np.inf
min_cells = 5

# read counts
adata = spatialdata_io.xenium(
    path,
    cells_as_circles=False,
    cells_boundaries=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
)["table"]


# preprocess (QC filters only)
# resolvi requires at least 5 counts in each cell
preprocessing.preprocess(
    adata,
    normalize=False,
    log1p=False,
    scale="none",
    pca=False,
    umap=False,
    save_raw=False,
    min_counts=min_counts,
    min_genes=min_features,
    max_counts=max_counts,
    max_genes=max_features,
    min_cells=min_cells,
)


scvi.external.RESOLVI.setup_anndata(
    adata, labels_key=None, layer=None, prepare_data_kwargs={"spatial_rep": "spatial"}
)
resolvi = scvi.external.RESOLVI(adata, semisupervised=False)
resolvi.train(max_epochs=50)

samples_corr = resolvi.sample_posterior(
    model=resolvi.module.model_corrected,
    return_sites=["px_rate"],
    summary_fun={"post_donor_q50": np.median},
    num_samples=3,
    summary_frequency=30,
)
samples_corr = pd.DataFrame(samples_corr).T

samples_proportions = resolvi.sample_posterior(
    model=resolvi.module.model_residuals,
    return_sites=["mixture_proportions"],
    summary_fun={"post_donor_means": np.mean},
    num_samples=3,
    summary_frequency=100,
)
samples_proportions = pd.DataFrame(samples_proportions).T

samples_corr = pd.DataFrame(
    samples_corr.loc["post_donor_q50", "px_rate"],
    index=adata.obs_names,
    columns=adata.var_names,
)
samples_proportions = pd.DataFrame(
    samples_proportions.loc["post_donor_means", "mixture_proportions"],
    index=adata.obs_names,
    columns=["true_proportion", "diffusion_proportion", "background_proportion"],
)


INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/segger/NSCLC/lun
         g/1GQ9/1GQ9/normalised_results/outs/cell_feature_matrix.h5                                                
Removed 1473  cells...
Removed 0  genes...
Transferring data to GPU...
Transferring data back to CPU...
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:158: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scanpy/neighbors/__init__.py:704: FutureWarning: `transformer='rapids'` is deprecated. Use `rapids_singlecell.tl.neighbors` instead.
  warn(msg, FutureWarning)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scvi/data/fields/_dataframe_field.py:187: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has f

Training:   0%|          | 0/50 [00:00<?, ?it/s]

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch:   0%|          | 0/78 [00:00<?, ?it/s]

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch:   0%|          | 0/78 [00:00<?, ?it/s]